# Librerias a utilizar

In [1]:
# librerias para manipulacion de archivos
import os
import glob

# librerias para manipulacion de imagenes
import cv2 as cv
import numpy as np

# librerias de utilidad
import hashlib

# Realizar primer procesamiento 
---
### Eliminacion de duplicados
La función **hash_archivo** calcula el hash MD5 del contenido de un archivo. Luego, el código recorre todos los archivos en el directorio, calculando el hash de cada archivo con las extensiones especificadas (jpg, jpeg, png, txt). Si el hash de un archivo ya existe en el diccionario hashes, se considera un duplicado y se elimina. Si el hash no existe, se agrega al diccionario.

### procesamiento de archivos de anotaciones e imagenes
La función **process_annotations** recorre los archivos de anotaciones, buscando las correspondientes imágenes, y convierte las anotaciones al formato YOLO si es necesario. También elimina los archivos de anotaciones si no se encuentra la imagen correspondiente. 

La función **process_annotations** tambien utiliza:
- La función **is_yolo_format** verifica si una línea de anotación está en el formato YOLO adecuado. 
- La función **convert_to_yolo_format** convierte las anotaciones que no están en el formato YOLO a dicho formato, calculando los valores del centro, ancho y alto de las bounding boxes a partir de las coordenadas de las esquinas. 

La función **validate_images_and_annotations** verifica que cada imagen tenga su archivo de anotación y viceversa, eliminando cualquier archivo huérfano que no tenga su correspondiente par. 

### **NOTA**: las imagenes deben descargarse manualmente desde el siguiente link:

*https://drive.google.com/drive/folders/1Ad7wlGabhY_QiTxzWQBVxbz0E8PqmUoK?usp=drive_link*

### Una vez descargadas solo se debe mover a la carpeta dentro de la carpeta truco_app y ejecutar las celdas que siguen.

In [2]:
# Ruta a la carpeta que contiene las imágenes y archivos .txt
DATA_PATH = "./fotos_textos_data"

### Eliminacion de duplicados

In [3]:
def hash_archivo(file_path):
    """Calcula el hash MD5 del contenido de un archivo."""
    hasher = hashlib.md5()
    with open(file_path, 'rb') as afile:
        buf = afile.read()
        hasher.update(buf)
    return hasher.hexdigest()

# Diccionario para almacenar hashes y sus rutas de archivo correspondientes
hashes = {}

# Recorrer todos los archivos en el directorio
for root, dirs, files in os.walk(DATA_PATH):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png', '.txt')):  # Filtra por extensiones deseadas
            ruta_completa_archivo = os.path.join(root, file)
            archivo_hash = hash_archivo(ruta_completa_archivo)
            
            if archivo_hash in hashes:
                # Si el hash ya existe, eliminar el archivo duplicado
                print(f'Eliminando duplicado: {ruta_completa_archivo}')
                os.remove(ruta_completa_archivo)
            else:
                # Si el hash no existe, agregarlo al diccionario
                hashes[archivo_hash] = ruta_completa_archivo

print("Eliminación de duplicados completada.")

Eliminación de duplicados completada.


### procesamiento de archivos.txt e imagenes

In [4]:
def is_yolo_format(line):
    parts = line.strip().split()
    if len(parts) == 5:
        try:
            int(parts[0])
            [float(coord) for coord in parts[1:]]
            return True
        except ValueError:
            return False
    return False

def convert_to_yolo_format(annotation_lines, image_w, image_h):
    yolo_lines = []
    for line in annotation_lines:
        parts = line.strip().split()
        if len(parts) > 5:
            class_id = parts[0]
            points = [float(p) for p in parts[1:]]
            points = np.array(points).reshape(-1, 2)
            
            x_min = np.min(points[:, 0]) * image_w
            y_min = np.min(points[:, 1]) * image_h
            x_max = np.max(points[:, 0]) * image_w
            y_max = np.max(points[:, 1]) * image_h

            x_center = (x_min + x_max) / 2 / image_w
            y_center = (y_min + y_max) / 2 / image_h
            width = (x_max - x_min) / image_w
            height = (y_max - y_min) / image_h

            yolo_line = f"{class_id} {x_center} {y_center} {width} {height}\n"
        else:
            yolo_line = line

        yolo_lines.append(yolo_line)
    return yolo_lines

def process_annotations(data_path):
    label_files = glob.glob(f"{data_path}/*.txt")

    for label_file in label_files:
        base_name = os.path.basename(label_file).split(".")[0]
        image_path = None

        for ext in ["jpg", "png", "jpeg"]:
            possible_path = os.path.join(data_path, f"{base_name}.{ext}")
            if os.path.exists(possible_path):
                image_path = possible_path
                break

        if not image_path:
            print(f"Imagen no encontrada para {label_file}, eliminando {label_file}")
            os.remove(label_file)
            continue

        image = cv.imread(image_path)
        if image is None:
            print(f"No se pudo leer la imagen: {image_path}, eliminando {label_file}")
            os.remove(label_file)
            continue

        image_h, image_w, _ = image.shape

        with open(label_file, "r") as file:
            lines = file.readlines()

        yolo_lines = []
        for line in lines:
            if is_yolo_format(line):
                yolo_lines.append(line)
            else:
                yolo_lines.extend(convert_to_yolo_format([line], image_w, image_h))

        with open(label_file, "w") as file:
            file.writelines(yolo_lines)

        print(f"Procesado {label_file} - Transformación aplicada correctamente")

def validate_images_and_annotations(data_path):
    image_files = []
    for ext in ["jpg", "png", "jpeg"]:
        image_files.extend(glob.glob(f"{data_path}/*.{ext}"))

    label_files = glob.glob(f"{data_path}/*.txt")

    image_basenames = {os.path.splitext(os.path.basename(f))[0] for f in image_files}
    label_basenames = {os.path.splitext(os.path.basename(f))[0] for f in label_files}

    images_without_labels = image_basenames - label_basenames
    labels_without_images = label_basenames - image_basenames

    for basename in images_without_labels:
        for ext in ["jpg", "png", "jpeg"]:
            image_path = os.path.join(data_path, f"{basename}.{ext}")
            if os.path.exists(image_path):
                print(f"Eliminando imagen sin etiqueta: {image_path}")
                os.remove(image_path)

    for basename in labels_without_images:
        label_path = os.path.join(data_path, f"{basename}.txt")
        if os.path.exists(label_path):
            print(f"Eliminando etiqueta sin imagen: {label_path}")
            os.remove(label_path)

In [5]:
# Primero validar imágenes y anotaciones
validate_images_and_annotations(DATA_PATH)

# Procesar imagenes y anotaciones
process_annotations(DATA_PATH)

Eliminando imagen sin etiqueta: ./fotos_textos_data\43265_Lucas_Demarré_04.jpg
Eliminando imagen sin etiqueta: ./fotos_textos_data\43265_Lucas_Demarré_29.jpg
Eliminando imagen sin etiqueta: ./fotos_textos_data\43265_Lucas_Demarré_30.jpg
Eliminando imagen sin etiqueta: ./fotos_textos_data\43265_Lucas_Demarré_13.jpg
Eliminando imagen sin etiqueta: ./fotos_textos_data\43265_Lucas_Demarré_18.jpg
Eliminando imagen sin etiqueta: ./fotos_textos_data\43265_Lucas_Demarré_23.jpg
Eliminando imagen sin etiqueta: ./fotos_textos_data\43265_Lucas_Demarré_10.jpg
Eliminando imagen sin etiqueta: ./fotos_textos_data\43265_Lucas_Demarré_03.jpg
Eliminando imagen sin etiqueta: ./fotos_textos_data\43265_Lucas_Demarré_25.jpg
Eliminando imagen sin etiqueta: ./fotos_textos_data\43265_Lucas_Demarré_24.jpg
Eliminando imagen sin etiqueta: ./fotos_textos_data\43265_Lucas_Demarré_09.jpg
Eliminando imagen sin etiqueta: ./fotos_textos_data\43265_Lucas_Demarré_22.jpg
Eliminando imagen sin etiqueta: ./fotos_textos_d